In [1]:
import pandas as pd
import numpy as np
import datetime
import os
print(datetime.datetime.now())

2020-08-12 14:41:29.192855


In [2]:
df_CN_ids=pd.read_csv("./notexposed_control_idlist.csv",usecols=['customer_id_hashed'],dtype=str)
df_TE_ids=pd.read_csv("./exposed_test_idlist.csv",usecols=['customer_id_hashed'],dtype=str)
df_CN_ids['group']="CN"
df_TE_ids['group']="TE"

df_ids_included=df_CN_ids.append(df_TE_ids)
print(df_ids_included.shape,df_ids_included['customer_id_hashed'].nunique())
df_total_ids=df_ids_included.groupby("group")['customer_id_hashed'].nunique().to_frame().reset_index()

(1010014, 2) 1010014


,group,customer_id_hashed
0,CN,585058
1,TE,424956


In [3]:
df_ids_included.head(2)

,customer_id_hashed,group
0,2573f2045385e11d9e39e1927efa5579a1a929e4045449...,CN
1,9d6cb850368a834e0150de40ce061d5b76d9aa332bc129...,CN


In [4]:
df_total_ids.head(2)

,group,customer_id_hashed
0,CN,585058
1,TE,424956


In [5]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

list_pos_q1=list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/"))
list_pos_q1=[x for x in list_pos_q1 if "dailysales" in x.lower()]
list_pos_q1=[x for x in list_pos_q1 if x.split("_weeks/MediaStorm_")[1][:10]>="2020-02-02"]
list_pos_q1=[x for x in list_pos_q1 if x.split("_weeks/MediaStorm_")[1][:10]<="2020-05-02"]
print(len(list_pos_q1))
list_pos_q1.sort()
df_q1_pos_files=pd.DataFrame({"file_path":list_pos_q1})
df_q1_pos_files['week_end_dt']=df_q1_pos_files['file_path'].apply(lambda x: x.split("_weeks/MediaStorm_")[1][:10])
df_q1_pos_files['period']=np.where(df_q1_pos_files['week_end_dt']>="2020-03-15","During_COVID","Pre_COVID")

13


In [6]:
df_period_summary_by_week=pd.DataFrame()
df_ids_both_periods_Q1=pd.DataFrame()
agg_func_sales_id={"customer_id_hashed":"nunique","item_transaction_amt":"sum"}
i=0
for period, df_group in df_q1_pos_files.groupby("period"):
    for ind,row in df_group.iterrows():
        file=row['file_path']
        week_end_dt=row['week_end_dt']
        
        df=pd.read_table(file,sep="|")
        df=pd.merge(df_ids_included,df,on="customer_id_hashed",how="inner")
        df_sales=df.groupby(['group'])['customer_id_hashed','item_transaction_amt'].agg(agg_func_sales_id).reset_index().rename(columns={"customer_id_hashed":"unique_shopper","item_transaction_amt":"sales"})
        df_trans=df[['customer_id_hashed','group','location_id','transaction_dt','transaction_id']].drop_duplicates()
        df_trans=df_trans.groupby(['group'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"transactions"})
        
        df_ids=df[['customer_id_hashed','group']].drop_duplicates()
        df_ids['period']=period
        
        df=pd.merge(df_sales,df_trans,on=["group"])
        df.insert(0,'period',period)
        df.insert(1,'week_end_dt',week_end_dt)
    
        df_period_summary_by_week=df_period_summary_by_week.append(df)
        df_ids_both_periods_Q1=df_ids_both_periods_Q1.append(df_ids)
        i+=1
        print(i,week_end_dt)
df_ids_both_periods_Q1=df_ids_both_periods_Q1.drop_duplicates()
        

1 2020-03-21
2 2020-03-28
3 2020-04-04
4 2020-04-11
5 2020-04-18
6 2020-04-25
7 2020-05-02
8 2020-02-08
9 2020-02-15
10 2020-02-22
11 2020-02-29
12 2020-03-07
13 2020-03-14


In [7]:
df_ids_both_periods_Q1.shape,df_ids_both_periods_Q1['customer_id_hashed'].nunique(),df_ids_both_periods_Q1['period'].unique()

((407857, 3), 312756, array(['During_COVID', 'Pre_COVID'], dtype=object))

In [8]:
df_ids_both_periods_Q1_summary=df_ids_both_periods_Q1.groupby(['period','group'])['customer_id_hashed'].nunique().to_frame().reset_index()

In [9]:
writer=pd.ExcelWriter("./BL_Q1_rewards_matched_in_1_to_1_JL_%s.xlsx"%str(datetime.datetime.now().date()),engine="xlsxwriter")
df_ids_both_periods_Q1_summary.to_excel(writer,"shopper_count_summary",index=False)
df_period_summary_by_week.to_excel(writer,"performance_by_week",index=False)
df_total_ids.to_excel(writer,"total_matched_ids",index=False)
writer.save()

In [10]:
df_q1_pos_files

,file_path,week_end_dt,period
0,/home/jian/BigLots/2020_by_weeks/MediaStorm_20...,2020-02-08,Pre_COVID
1,/home/jian/BigLots/2020_by_weeks/MediaStorm_20...,2020-02-15,Pre_COVID
2,/home/jian/BigLots/2020_by_weeks/MediaStorm_20...,2020-02-22,Pre_COVID
3,/home/jian/BigLots/2020_by_weeks/MediaStorm_20...,2020-02-29,Pre_COVID
4,/home/jian/BigLots/2020_by_weeks/MediaStorm_20...,2020-03-07,Pre_COVID
5,/home/jian/BigLots/2020_by_weeks/MediaStorm_20...,2020-03-14,Pre_COVID
6,/home/jian/BigLots/2020_by_weeks/MediaStorm_20...,2020-03-21,During_COVID
7,/home/jian/BigLots/2020_by_weeks/MediaStorm_20...,2020-03-28,During_COVID
8,/home/jian/BigLots/2020_by_weeks/MediaStorm_20...,2020-04-04,During_COVID
9,/home/jian/BigLots/2020_by_weeks/MediaStorm_20...,2020-04-11,During_COVID
